In [2]:
import os
from bs4 import BeautifulSoup
import re

from selenium import webdriver
from selenium.webdriver.common.by import By
import time

import requests


In [ ]:

url_base="https://www.freshproduce.com/"

url = "https://www.freshproduce.com/resources/"

#Carpeta para las cosas
if not os.path.exists("url_sixto"):
    os.makedirs("url_sixto")

#extraemos la informacion de la pagina de recursos y la guardamos en un .txt
! curl -A "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_2_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.2 Safari/605.1.15" -o url_sixto/resources.txt $url

#Determinamos las categorias que nos interesan
categories = ["global-trade", "technology", "food-safety"]

#Leemos el html de la página inicial y parseamos con BeautifulSoup
with open("url_sixto/resources.txt", "r") as file:
    content = file.read()
    soup = BeautifulSoup(content, "html.parser")
    cathegory_links = []
    for category in categories:
        #Nos quedamos con los URL's que nos mandan las categorias deseadas
        category_links = soup.find_all("a", href=re.compile(f"/{category}/"))
        for link in category_links:
            href = link.get("href")
            #Quitamos URL's indeseables:
            if href and href.startswith("/resources/"):
                if href.endswith(f"{category}/"):
                    # Construct the full URL
                    full_url = url_base + href
                    cathegory_links.append(full_url)


#elimimamos duplicados
cathegory_links = list(set(cathegory_links))

#Condición de ser "Article"
filtro="?pageNumber=0&filteredCategories=Article"


#Creamos y nombramos el csv
with open(f"url_sixto/fresh_produce.csv", "a") as f :
    f.write(f"Title,")
    f.write(f"URL,")
    f.write(f"Category,")
    f.write(f"FullArticleText ")
    f.write("\n")


#de la página de cada categoria extraemos los enlaces de sus artículos
for link in cathegory_links:
    #<title> Technology</title>
    category_name = link.split("/")[-2].replace('-', ' ')
    print(f"Category: {category_name}\n")

    #llamamos a la api para extraer los enlaces de la página (pero primero le pedimos que solamente nos dé artículos)
    driver = webdriver.Chrome()  # o webdriver.Firefox()
    full_link= link+filtro
    driver.get(full_link) 

    #Damos tiempo para que carge la pagina (evita posibles problemas)
    time.sleep(2)

    #Extraemos los links de los botones LEARN MORE
    links_learnmore = []
    for a in driver.find_elements(By.PARTIAL_LINK_TEXT, "LEARN MORE"):
        links_learnmore.append(a.get_attribute("href"))

    #elimiamos el primer URL (que corresponde a una ventana deslizante y no nos interesa)
    if links_learnmore:
        links_learnmore.pop(0)


    #Ciclo para llenar el .csv
    with open(f"url_sixto/fresh_produce.csv", "a") as f :
        
        for link1 in links_learnmore:
            #Para extraer y darle formato al texto:
            #Extraemos el html de link1
            url = link1
            html_link1 = requests.get(url)
            if html_link1.status_code == 200:
                html_content = html_link1.text
                print(html_content)
            else:
                print(f"Failed to retrieve page")


            soup = BeautifulSoup(html_content, 'html.parser')
            #Extraemos el título (quitamos comas para evitar saltos de celda en el .csv)
            meta_tag = soup.find('meta', attrs={'name': 'title'})
            if meta_tag and 'content' in meta_tag.attrs:
                title = meta_tag['content'].replace(',', '')
                print(title)
            else:
                print("Meta tag not found or content attribute missing.")

            #Buscamos el texto del artículo en el HTLM
            target_divs = soup.select('div.score-content-spot.rich-text[data-epi-type="content"]')
            #Quitamos la dirección que aparece al final de cada artículo
            article_bodies = [div.get_text(separator=' ', strip=True) for div in target_divs[:-1]]
            combined_article_body = " ".join(article_bodies)
            print(combined_article_body)

            #Escribimos lo extraído en el csv
            f.write(f"{title}, ")
            f.write(f"{link1}, ")
            f.write(f"{category_name}, ")
            f.write(f"{combined_article_body.replace(',', '')} \n")
        #f.write("\n")

    for link1 in links_learnmore:
        print(link1)
        

    driver.quit()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  112k    0  112k    0     0   167k      0 --:--:-- --:--:-- --:--:--  167k
Category: global trade





<!DOCTYPE html>
<html lang="en" class="">
<head>
    <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=5" />
    <meta name="author" content="" />

    <link rel="apple-touch-icon" sizes="180x180" href="/favicons/apple-touch-icon.png?v=8.0.0.0">
    <link rel="icon" type="image/png" sizes="32x32" href="/favicons/favicon-32x32.png?v=8.0.0.0">
    <link rel="icon" type="image/png" sizes="16x16" href="/favicons/favicon-16x16.png?v=8.0.0.0">
    <link rel="manifest" href="/favicons/site.webmanifest?v=8.0.0.0">
    <link rel="mask-icon" href="/favicons/safari-pinned-tab.svg?v=8.0.0.0" color="#5bbad5">
    <meta name